# `vaex` @ PyData London 2019
## New York Taxi Dataset (2009-2015): Exploratory Data Analysis and Machine Learning example

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
import vaex
from vaex.ui.colormaps import cm_plusmin
import warnings; warnings.simplefilter('ignore')


import pylab as plt
import numpy as np
import seaborn as sns

In [ ]:
!ls -lh /data

In [ ]:
df_gaia = vaex.open('/data/gaia-dr2-sort-by-source_id.hdf5')

In [ ]:
df_gaia

### Open the dataset

In [ ]:
# Opens the data
df = vaex.open('/data/yellow_taxi_2009_2015_f32.hdf5')

# Check length
print(f'Number of samples in the data: {len(df):,}')

In [ ]:
# View a portion of the dataset
df

### Split the data into train & test sets

In [ ]:
# Train / test split (by date)
df_train = df[:1_026_944_937]
df_test = df[1_026_944_937:]

print(f'Number of samples in the training set: {len(df_train):,}')
print(f'Number of samples in the test set:       {len(df_test):,}')

# Check if the lengths of the datasets match
assert len(df) == len(df_test) + len(df_train)

### Basic view in the contents of the data

In [ ]:
# Basic description about the training dataset
df_train.describe()
# 9 min

### Remove missing data

In [ ]:
# Drop NANs
df_train = df_train.dropna(column_names=['dropoff_latitude', 'dropoff_longitude', 'pickup_latitude'])

### Abnormal number of passengers

In [ ]:
# Number of passengers
df_train.passenger_count.value_counts(progress=True)

In [ ]:
# Filter abnormal number of passengers
df_train = df_train[(df_train.passenger_count>0) & (df_train.passenger_count<7)]

### Clean up distance values

In [ ]:
plt.figure(figsize=(8,4))
df_train.plot1d('trip_distance', limits='minmax', f='log1p')
plt.show()

In [ ]:
# How many trips have 0.0 distance?
(df_train.trip_distance==0).astype('int').sum()

In [ ]:
# What is the largest distance?
_ = df_train.trip_distance.max()

print(_, 'miles.')

print('This is %3.1f times larger than the distance between the Earth and the Moon!' % (_ / 238_900))
print('or')
print('This is %1.1f the distance to Mars!' % (_ / 33_900_000))

In [ ]:
plt.figure(figsize=(8,4))
df_train.plot1d('trip_distance', limits=[0, 20], f=None)
plt.show()

In [ ]:
# Filter negative and too large distances
df_train = df_train[(df_train.trip_distance>0) & (df_train.trip_distance<10)]

### What _is_ New York City really?

In [ ]:
# Interactively plot the pickup locations
df_train.plot_widget(df_train.pickup_longitude, 
                     df_train.pickup_latitude, 
                     shape=512, 
                     f='log1p', 
                     colormap='plasma', 
                     limits='minmax')

In [ ]:
# Define the NYC boundaries
long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90

In [ ]:
# Make a selection based on the boundaries
df_train = df_train[(df_train.pickup_longitude > long_min)  & (df_train.pickup_longitude < long_max) & \
        (df_train.pickup_latitude > lat_min)    & (df_train.pickup_latitude < lat_max) & \
        (df_train.dropoff_longitude > long_min) & (df_train.dropoff_longitude < long_max) & \
        (df_train.dropoff_latitude > lat_min)   & (df_train.dropoff_latitude < lat_max)]

### Create some features

In [ ]:
# Speed (miles per hour)
df_train['trip_speed_mph'] = df_train.trip_distance / ((df_train.dropoff_datetime - df_train.pickup_datetime) / np.timedelta64(1, 'h'))

# Time in transit (minutes)
df_train['trip_duration_min'] = (df_train.dropoff_datetime - df_train.pickup_datetime) / np.timedelta64(1, 'm')

# fare divided by distance
df_train['fare_by_distance'] = (df_train.fare_amount / df_train.trip_distance).jit_numba()

### More filters: Trip duration

In [ ]:
plt.figure(figsize=(8,4))
df_train.plot1d('trip_duration_min', f='log1p', limits='minmax')
plt.ylabel('log(count + 1)')
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
df_train.plot1d('trip_duration_min', f='log1p', limits=[0, 1000])
plt.ylabel('log(count + 1)')
plt.show()

In [ ]:
# Filter, keep durations that are within 2 hours
df_train = df_train[(df_train.trip_duration_min>0) & (df_train.trip_duration_min<120)]

### Create some date/time features

In [ ]:
# Daily activities
df_train['pu_hour'] = df_train.pickup_datetime.dt.hour
df_train['pu_day_of_week'] = df_train.pickup_datetime.dt.dayofweek
df_train['pu_month'] = df_train.pickup_datetime.dt.month - 1
df_train['pu_is_weekend'] = (df_train.pu_day_of_week>=5).astype('int')

# lists to help with the labeling
weekday_names_list = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
month_names_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
# Treat these columns as label/ordinal encoded values
df_train.categorize(column='pu_hour')
df_train.categorize(column='pu_day_of_week')

In [ ]:
# Number of pick-ups per hour for a given day of the week
df_train.plot('pu_hour', 'pu_day_of_week', colorbar=True, colormap=cm_plusmin, figsize=(15, 5))
plt.xticks(np.arange(24), np.arange(24))
plt.yticks(np.arange(7), weekday_names_list)
plt.show()

### Groupby examples

In [ ]:
df_per_hour = df_train.groupby(by=df_train.pu_hour).agg({'tip_amount': 'mean',
                                                         'trip_speed_mph': 'mean',
                                                        })

# Display the grouped DataFrame
df_per_hour

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(121)
sns.barplot(x=df_per_hour.pu_hour.values, y=df_per_hour.tip_amount.values)
plt.title('Mean tip amount')
plt.xlabel('hour of day')
plt.ylabel('mean tip amount')

plt.subplot(122)
sns.barplot(x=df_per_hour.pu_hour.values, y=df_per_hour.trip_speed_mph.values)
plt.title('Mean trip speed')
plt.xlabel('hour of day')
plt.ylabel('mean trip speed [miles per hour]')


plt.tight_layout()
plt.show()

In [ ]:
# All the filtering, in case something went wrong
import vaex
import numpy as np
import pylab as plt
df = vaex.open('/data/yellow_taxi_2009_2015_f32.hdf5')
magic_row_nr = 1_026_944_937
df_train, df_test = df[:magic_row_nr], df[magic_row_nr:]
df_train = df_train.dropna(column_names=['dropoff_latitude', 'dropoff_longitude', 'pickup_latitude'])
df_train = df_train[(df_train.passenger_count>0) & (df_train.passenger_count<7)]
df_train = df_train[(df_train.trip_distance>0) & (df_train.trip_distance<10)]
# Define the NYC boundaries
long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90
# Make a selection based on the boundaries
df_train = df_train[(df_train.pickup_longitude > long_min)  & (df_train.pickup_longitude < long_max) & \
        (df_train.pickup_latitude > lat_min)    & (df_train.pickup_latitude < lat_max) & \
        (df_train.dropoff_longitude > long_min) & (df_train.dropoff_longitude < long_max) & \
        (df_train.dropoff_latitude > lat_min)   & (df_train.dropoff_latitude < lat_max)]
df_train['pu_hour'] = df_train.pickup_datetime.dt.hour
df_train['pu_day_of_week'] = df_train.pickup_datetime.dt.dayofweek
df_train['pu_month'] = df_train.pickup_datetime.dt.month - 1
df_train['pu_is_weekend'] = (df_train.pu_day_of_week>=5).astype('int')
df_train['trip_speed_mph'] = df_train.trip_distance / ((df_train.dropoff_datetime - df_train.pickup_datetime) / np.timedelta64(1, 'h'))
df_train['trip_duration_min'] = (df_train.dropoff_datetime - df_train.pickup_datetime) / np.timedelta64(1, 'm')
df_train['fare_by_distance'] = (df_train.fare_amount / df_train.trip_distance)


## Predictive modelling example: predict the likely duration of a taxi trip

In [ ]:
df_train

In [ ]:
import vaex.ml

In [ ]:
# arc-distance in miles
def arc_distance(theta_1, phi_1, theta_2, phi_2):
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    distance = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return distance * 3958.8

# distance London - Utrecht [miles]
arc_distance(51.5069797, -0.1295992, 52.0842715, 5.0124523)

In [ ]:
# Add the arc-distance in miles as a virtual column
df_train['arc_distance_miles_numpy'] = arc_distance(df_train.pickup_longitude, df_train.pickup_latitude, 
                                              df_train.dropoff_longitude, df_train.dropoff_latitude)

In [ ]:
df_train['arc_distance_miles_numpy'].sum(progress=True)

In [ ]:
df_train['arc_distance_miles_cuda'] = df_train['arc_distance_miles_numpy'].jit_cuda()

In [ ]:
df_train['arc_distance_miles_cuda'].sum(progress=True)

In [ ]:
# choose GPU or CPU
df_train['arc_distance_miles'] = df_train['arc_distance_miles_cuda']
# df_train['arc_distance_miles'] = df_train['arc_distance_miles_numpy']

In [ ]:
# direction of travel in degrees
def direction_angle(theta_1, phi_1, theta_2, phi_2):
    dtheta = theta_2 - theta_1
    dphi = phi_2 - phi_1
    radians = np.arctan2(dtheta, dphi)
    return np.rad2deg(radians)

# The direction of travel
df_train['direction_angle'] = direction_angle(df_train.pickup_longitude, df_train.pickup_latitude, 
                                           df_train.dropoff_longitude, df_train.dropoff_latitude).jit_numba()

In [ ]:
# Examine the train DataFrame at this point 
df_train.head(10)

### Encoding and transforming of features

In [ ]:
# PCA of the pickup and dropoff locations - helps to "straighten out" the coordinates

# pickup transformations
pca_pu = vaex.ml.PCA(features=['pickup_longitude', 'pickup_latitude'], n_components=2)
df_train = pca_pu.fit_transform(df_train)

# dropoff transformations
pca_do = vaex.ml.PCA(features=['dropoff_longitude', 'dropoff_latitude'], n_components=2)
df_train = pca_do.fit_transform(df_train)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(121)
plt.title('pickup - original')
df_train.plot(df_train.pickup_longitude, df_train.pickup_latitude,
           colormap='plasma', f='log1p', shape=256, colorbar=False)

plt.subplot(122)
plt.title('pickup - PCA transformed')
df_train.plot(df_train.PCA_0, df_train.PCA_1,
           colormap='plasma', f='log1p', shape=256, colorbar=False)

plt.tight_layout()
plt.show()

In [ ]:
# Inspect the payment_type
df_train.payment_type.str.lower().value_counts(progress=True)

Inspect the _payment_\__type_
From the documentation provided:
- 1 = Credit card
- 2 = Cash
- 3 = No charge
- 4 = Dispute
- 5 = Unknown
- 6 = Voided trip

In [ ]:
# Define a mapping dictionary
map_payment_type = {'csh': 2, 'crd': 1, 'cash': 2, '1': 1, 'cas': 2, '2': 2, 'credit': 1, 'cre': 1, 'unk': 5, 
                    'noc': 3, 'no charge': 3, '3':3, 'dis': 4, 'no ': 3, '4': 4, 'dispute': 4, 'na ': 5, '5':5}

df_train['payment_type_'] = df_train.payment_type.str.lower().map(map_payment_type, 
                                                                  default_value=-1, 
                                                                  allow_missing=True)

In [ ]:
# inspect the DataFrame
df_train.head(5)

### Setting up the predictor - `LightGBM`

In [ ]:
features_lgbm = ['passenger_count', 'trip_distance', 'rate_code', 'pu_hour', 'pu_day_of_week', 'pu_is_weekend', 
                 'arc_distance_miles', 'direction_angle', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'payment_type_']


# the target
target = 'trip_duration_min'

In [ ]:
# Import the modeling library
import vaex.ml.lightgbm # vaex.ml also supports XGBoost, CatBoost, scikit-learn, annoy, more to come)

In [ ]:
# parameters - standard lightgbm options
params = {
    'learning_rate': 0.1,       
    'max_depth': 5,             
    'colsample_bytree': 0.8,
    'subsample': 0.8,           
    'reg_lambda': 1,            
    'reg_alpha': 0,             
    'min_child_weight': 1,      
    'objective': 'regression',  
    'random_state': 42,         
    'n_jobs': -1} 

# Instantiate the model object
booster = vaex.ml.lightgbm.LightGBMModel(features=features_lgbm, params=params, num_boost_round=100)

# Take small part of the training set to we can do the training in real time fast
df_train_mini = df_train[:1_000_000]

# Fit the model object
booster.fit(df_train_mini, target=target)

print('Training completed!')

In [ ]:
# Check performance on the training set - in reality one needs to do proper (x)-validation

# Classical predict - get an in-memory array of the predictions
pred = booster.predict(df_train_mini)

# view the predictions
display(pred)

# Create a virtual column housing the predictions
df_train = booster.transform(df_train)

# view the DataFrame
df_train.head(5)

In [ ]:
# Check the performance
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_train_score = mean_absolute_error(df_train_mini.trip_duration_min.values, pred)
mse_train_score = mean_squared_error(df_train_mini.trip_duration_min.values, pred)

print('The mean absolute error is %2.3f' % mae_train_score)
print('The mean squared score is %2.3f' % mse_train_score)

### Second estimator?!

In [ ]:
# One hot encoding of categorical features
one_hot_enc = vaex.ml.OneHotEncoder(features=['payment_type_', 
                                              'pu_hour', 'pu_day_of_week', 'pu_month'], prefix='onehot_')

df_train = one_hot_enc.fit_transform(df_train)

In [ ]:
# Standard scale some of the numerical features
standard_scaler = vaex.ml.StandardScaler(features=['arc_distance_miles', 'direction_angle', 'trip_distance'])
df_train = standard_scaler.fit_transform(df_train)

In [ ]:
# Try a linear model
import vaex.ml.sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
# Specify which features to use
features_linear = ['PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'pu_is_weekend'] + \
                  [feature for feature in df_train.get_column_names() if 'standard_scaled_' in feature] + \
                  [feature for feature in df_train.get_column_names() if 'onehot_' in feature]

In [ ]:
# Instantiate the vaex-sklearn model 
linear_model = vaex.ml.sklearn.SKLearnPredictor(model=LinearRegression(copy_X=False, n_jobs=-1), 
                                                features=features_linear, prediction_name='linear_prediction')

# Agaom, take small part of the training so we can train in real time.
df_train_mini = df_train[:1_000_000]


# Fit the model object
linear_model.fit(df_train_mini, target=target)

print('Training completed!')

In [ ]:
# Check performance on the training set - in reality one needs to do proper (x)-validation

# Classical predict - get an in-memory array of the predictions
pred_linear = linear_model.predict(df_train_mini)

# view the predictions
display(pred_linear)

# Create a virtual column housing the predictions
df_train = linear_model.transform(df_train)

# view the DataFrame 
df_train.head(5)

In [ ]:
mae_train_score = mean_absolute_error(df_train_mini.trip_duration_min.values, pred_linear)
mse_train_score = mean_squared_error(df_train_mini.trip_duration_min.values, pred_linear)

print('The mean absolute error is %2.3f' % mae_train_score)
print('The mean squared score is %2.3f' % mse_train_score)

### Ensemble?!

In [ ]:
# Average the predictions from the Gradient Boosting and Linear models
df_train['final_prediction'] = (df_train.lightgbm_prediction + df_train.linear_prediction) / 2

In [ ]:
### Display the DataFrame
df_train.head(5)

In [ ]:
mae_train_score = mean_absolute_error(df_train[:1_000_000].trip_duration_min.values, 
                                      df_train[:1_000_000].final_prediction.values)
mse_train_score = mean_squared_error(df_train[:1_000_000].trip_duration_min.values, 
                                     df_train[:1_000_000].final_prediction.values)

print('The mean absolute error is %2.3f' % mae_train_score)
print('The mean squared score is %2.3f' % mse_train_score)

### So what about a pipeline?

## The `vaex` `state` – all the pipeline you need!

In [ ]:
# Save the state to disk
state = df_train.state_write('./taxi_ml_state.json')